understand this client requiremnt algorithm below and suggest me the best and easiest model to accurate execute the below funciton and after bulding the model save it in .pkl file and then write the code for this below program.


import necesssary libraries 

load crop and  fertilizer data 

user input = crop name, crop growth stage , water nutrietion = [water ph,water ec,water nitrogen,water phosphorus,water potassium,water calcium,water magnesium,water sulfur,water copper,water chlorine,water boron,water iron,water zinc,water manganese,water molybdenum,water nickel,water cobalt,water sodium]


input for findng the recommended fertilizer  = {recommended crop nutrition fing using user input crop.csv and cropname and crop growth stage} - water nutrietion (provide in user input)

this difference in nutrietion value we have to minimize , so now


divide the difference input into two parts one dor micro nutrients and another one for macro nutritients

use trained model on fertilizer dataset to find the two fertilizer of each micro nutritional difference and macro nutrient difference.


now after getting the name of the recommenced fertilizers , use fertilizer.csv to get more information on them,

now use that information and difference input values and try to calculate the recommended quanity in kg of that fertilizers to reduce the difference in water nutritional and ph ec content and crop recommened  values. 

give the estimate values, 

the values in crop data is per kg / 1000 liter water 
the values in fertilizer data is per kg / 1000 KG fertiizer

suggest the fertilzer which fulfill most the the elements deficiency in one go, if there are some elements missing in ones fertilizer but the crop need it , search for such particular fertilizer and calculate the amount we need to use in kg,


in the end return me the name of all the fertilizer , recommended quantity in which we have to use it, it solubility and toxicity levels , temperture at which we have to use it,  role and remark.




In [20]:
import pandas as pd

# Load the crop data
crop_data = pd.read_csv('crop_data.csv')

# Load the fertilizer data
fertilizer_data = pd.read_csv('./pro_fert_data.csv')

def get_nutrient_name(index):
    nutrient_names = ['crop_nitrogen', 'crop_phosphorus', 'crop_potassium', 'crop_calcium', 'crop_magnesium', 'crop_sulfur', 'crop_copper', 'crop_chlorine', 'crop_boron', 'crop_iron', 'crop_zinc', 'crop_manganese', 'crop_molybdenum', 'crop_nickel']
    return nutrient_names[index]

def calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition):
    # Get the recommended nutrition values for the crop
    crop_row = crop_data[(crop_data['crop_name'] == crop_name) & (crop_data['crop_growth_stage'] == crop_growth_stage)]
    
    if crop_row.empty:
        print(f"No data found for {crop_name} in {crop_growth_stage} stage")
        return []
    
    recommended_nutrition = crop_row.iloc[0, 5:].values  # Skip the 'crop_id', 'crop_name', 'crop_growth_stage', 'crop_type', 'crop_temp', 'crop_humidity', 'crop_ph' columns
    
    # Convert recommended_nutrition to float if possible
    recommended_nutrition = [float(x) if isinstance(x, str) and x.replace('.', '', 1).isdigit() else x for x in recommended_nutrition]
    
    # Calculate the difference between user-input water nutrition values and recommended values
    nutrient_deficiencies = []
    for i in range(14):
        if isinstance(recommended_nutrition[i], (int, float)) and isinstance(water_nutrition[i], (int, float)):
            deficiency = recommended_nutrition[i] - water_nutrition[i]
            nutrient_deficiencies.append((i, deficiency))
        else:
            nutrient_deficiencies.append((i, "Invalid input"))
    
    # Sort the deficiencies by absolute value in descending order
    numeric_deficiencies = [x for x in nutrient_deficiencies if isinstance(x[1], (int, float))]
    numeric_deficiencies.sort(key=lambda x: abs(x[1]), reverse=True)
    non_numeric_deficiencies = [x for x in nutrient_deficiencies if not isinstance(x[1], (int, float))]
    nutrient_deficiencies = numeric_deficiencies + non_numeric_deficiencies
    
    # Return the top 5 nutrient deficiencies
    return nutrient_deficiencies[:5]

def recommend_fertilizers(nutrient_deficiencies):
    recommended_fertilizers = []
    for deficiency in nutrient_deficiencies:
        nutrient_name = get_nutrient_name(deficiency[0]).replace('crop_', 'fert_')
        deficiency_value = deficiency[1]
        
        # Check if the nutrient_name exists in the fertilizer_data columns
        if nutrient_name in fertilizer_data.columns:
            # Search the fertilizer data for products that can fulfill this deficiency
            fertilizer_rows = fertilizer_data[fertilizer_data[nutrient_name] > 0]
            fertilizer_rows = fertilizer_rows.sort_values(by=nutrient_name, ascending=False)
            
            # Add the top 2 fertilizers to the recommended list
            for index, row in fertilizer_rows.head(2).iterrows():
                recommended_fertilizers.append((nutrient_name.replace('fert_', ''), row['fert_name'], row[nutrient_name]))
        else:
            print(f"No fertilizers found for {nutrient_name.replace('fert_', '')}")
    
    return recommended_fertilizers

def main():
    crop_name = "bokchoy"
    crop_growth_stage = "flowering"
    water_nutrition = [22.285, 281.586, 4.888, 41.023, 16.563, 7.896, 3.100, 1.883, 0.311, 0.297, 0.299, 0.317, 0.309, 0.297, 0.298, 0.298, 0.311, 0.290, 0.542, 1.417, 480.318]
    
    nutrient_deficiencies = calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition)
    recommended_fertilizers = recommend_fertilizers(nutrient_deficiencies)
    
    print("Recommended Fertilizers:")
    for fertilizer in recommended_fertilizers:
        print(f"{fertilizer[0]}: {fertilizer[1]} ({fertilizer[2]})")

if __name__ == "__main__":
    main()

Recommended Fertilizers:
phosphorus: nickelphosphate (773)
phosphorus: sodiumphosphate (772)
nitrogen: urea (812)
nitrogen: ammoniumnitrate (788)
potassium: sugarcanemolasses (797)
potassium: potassiumcarbonate (747)
calcium: calciumhydroxide (544)
calcium: calciumsilicate (508)
magnesium: magnesiumoxide (602)
magnesium: magnesiumhydroxide (434)


In [21]:
import pandas as pd

# Load the crop data
crop_data = pd.read_csv('crop_data.csv')

# Load the fertilizer data
fertilizer_data = pd.read_csv('./pro_fert_data.csv')

def get_nutrient_name(index):
    nutrient_names = ['crop_nitrogen', 'crop_phosphorus', 'crop_potassium', 'crop_calcium', 'crop_magnesium', 'crop_sulfur', 'crop_copper', 'crop_chlorine', 'crop_boron', 'crop_iron', 'crop_zinc', 'crop_manganese', 'crop_molybdenum', 'crop_nickel']
    return nutrient_names[index]

def calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition):
    # Get the recommended nutrition values for the crop
    crop_row = crop_data[(crop_data['crop_name'] == crop_name) & (crop_data['crop_growth_stage'] == crop_growth_stage)]
    
    if crop_row.empty:
        print(f"No data found for {crop_name} in {crop_growth_stage} stage")
        return []
    
    recommended_nutrition = crop_row.iloc[0, 5:].values  # Skip the 'crop_id', 'crop_name', 'crop_growth_stage', 'crop_type', 'crop_temp', 'crop_humidity', 'crop_ph' columns
    
    # Convert recommended_nutrition to float if possible
    recommended_nutrition = [float(x) if isinstance(x, str) and x.replace('.', '', 1).isdigit() else x for x in recommended_nutrition]
    
    # Calculate the difference between user-input water nutrition values and recommended values
    nutrient_deficiencies = []
    for i in range(14):
        if isinstance(recommended_nutrition[i], (int, float)) and isinstance(water_nutrition[i], (int, float)):
            deficiency = recommended_nutrition[i] - water_nutrition[i]
            nutrient_deficiencies.append((i, deficiency))
        else:
            nutrient_deficiencies.append((i, "Invalid input"))
    
    # Sort the deficiencies by absolute value in descending order
    numeric_deficiencies = [x for x in nutrient_deficiencies if isinstance(x[1], (int, float))]
    numeric_deficiencies.sort(key=lambda x: abs(x[1]), reverse=True)
    non_numeric_deficiencies = [x for x in nutrient_deficiencies if not isinstance(x[1], (int, float))]
    nutrient_deficiencies = numeric_deficiencies + non_numeric_deficiencies
    
    # Return the top 5 nutrient deficiencies
    return nutrient_deficiencies[:5]

def recommend_fertilizers(nutrient_deficiencies):
    recommended_fertilizers = []
    for deficiency in nutrient_deficiencies:
        nutrient_name = get_nutrient_name(deficiency[0]).replace('crop_', 'fert_')
        deficiency_value = deficiency[1]
        
        # Check if the nutrient_name exists in the fertilizer_data columns
        if nutrient_name in fertilizer_data.columns:
            # Search the fertilizer data for products that can fulfill this deficiency
            fertilizer_rows = fertilizer_data[fertilizer_data[nutrient_name] > 0]
            fertilizer_rows = fertilizer_rows.sort_values(by=nutrient_name, ascending=False)
            
            # Add the top 2 fertilizers to the recommended list
            for index, row in fertilizer_rows.head(2).iterrows():
                recommended_fertilizers.append((nutrient_name.replace('fert_', ''), row['fert_name'], row[nutrient_name]))
        else:
            print(f"No fertilizers found for {nutrient_name.replace('fert_', '')}")
    
    return recommended_fertilizers

def select_top_fertilizers(recommended_fertilizers):
    macro_fertilizers = []
    micro_fertilizers = []
    ph_ec_fertilizers = []

    macro_nutrients = ['nitrogen', 'phosphorus', 'potassium', 'calcium', 'agnesium']
    micro_nutrients = ['sulfur', 'copper', 'chlorine', 'boron', 'iron', 'zinc', 'anganese', 'olybdenum', 'nickel', 'cobalt', 'odium']

    for fertilizer in recommended_fertilizers:
        if fertilizer[0] in macro_nutrients:
            macro_fertilizers.append(fertilizer)
        elif fertilizer[0] in micro_nutrients:
            micro_fertilizers.append(fertilizer)
        else:
            ph_ec_fertilizers.append(fertilizer)

    top_macro_fertilizers = sorted(macro_fertilizers, key=lambda x: x[2], reverse=True)[:2]
    top_micro_fertilizers = sorted(micro_fertilizers, key=lambda x: x[2], reverse=True)[:2]
    top_ph_ec_fertilizer = sorted(ph_ec_fertilizers, key=lambda x: x[2], reverse=True)[0]

    return top_macro_fertilizers + top_micro_fertilizers + [top_ph_ec_fertilizer]

def main():
    crop_name = "bokchoy"
    crop_growth_stage = "flowering"
    water_nutrition = [22.285, 281.586, 4.888, 41.023, 16.563, 7.896, 3.100, 1.883, 0.311, 0.297, 0.299, 0.317, 0.309, 0.297, 0.298, 0.298, 0.311, 0.290, 0.542, 1.417, 480.318]
    
    nutrient_deficiencies = calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition)
    recommended_fertilizers = recommend_fertilizers(nutrient_deficiencies)
    
    top_fertilizers = select_top_fertilizers(recommended_fertilizers)
    
    print("Top 5 Fertilizers:")
    for fertilizer in top_fertilizers:
        print(f"{fertilizer[0]}: {fertilizer[1]} ({fertilizer[2]})")

if __name__ == "__main__":
    main()

Top 5 Fertilizers:
nitrogen: urea (812)
potassium: sugarcanemolasses (797)
magnesium: magnesiumoxide (602)


In [43]:
import pandas as pd

# Load the crop data
crop_data = pd.read_csv('crop_data.csv')

# Load the fertilizer data
fertilizer_data = pd.read_csv('./pro_fert_data.csv')

def get_nutrient_name(index):
    nutrient_names = ['crop_nitrogen', 'crop_phosphorus', 'crop_potassium', 'crop_calcium', 'crop_magnesium', 'crop_sulfur', 'crop_copper', 'crop_chlorine', 'crop_boron', 'crop_iron', 'crop_zinc', 'crop_manganese', 'crop_molybdenum', 'crop_nickel']
    return nutrient_names[index]

def calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition):
    # Get the recommended nutrition values for the crop
    crop_row = crop_data[(crop_data['crop_name'] == crop_name) & (crop_data['crop_growth_stage'] == crop_growth_stage)]
    
    if crop_row.empty:
        print(f"No data found for {crop_name} in {crop_growth_stage} stage")
        return []
    
    recommended_nutrition = crop_row.iloc[0, 5:].values  # Skip the 'crop_id', 'crop_name', 'crop_growth_stage', 'crop_type', 'crop_temp', 'crop_humidity', 'crop_ph' columns
    
    # Convert recommended_nutrition to float if possible
    recommended_nutrition = [float(x) if isinstance(x, str) and x.replace('.', '', 1).isdigit() else x for x in recommended_nutrition]
    
    # Calculate the difference between user-input water nutrition values and recommended values
    nutrient_deficiencies = []
    for i in range(14):
        if isinstance(recommended_nutrition[i], (int, float)) and isinstance(water_nutrition[i], (int, float)):
            deficiency = recommended_nutrition[i] - water_nutrition[i]
            nutrient_deficiencies.append((i, deficiency))
        else:
            nutrient_deficiencies.append((i, "Invalid input"))
    
    # Sort the deficiencies by absolute value in descending order
    numeric_deficiencies = [x for x in nutrient_deficiencies if isinstance(x[1], (int, float))]
    numeric_deficiencies.sort(key=lambda x: abs(x[1]), reverse=True)
    non_numeric_deficiencies = [x for x in nutrient_deficiencies if not isinstance(x[1], (int, float))]
    nutrient_deficiencies = numeric_deficiencies + non_numeric_deficiencies
    
    # Return the top 5 nutrient deficiencies
    return nutrient_deficiencies[:5]

def recommend_fertilizers(nutrient_deficiencies):
    recommended_fertilizers = []
    for deficiency in nutrient_deficiencies:
        nutrient_name = get_nutrient_name(deficiency[0]).replace('crop_', 'fert_')
        deficiency_value = deficiency[1]
        
        # Check if the nutrient_name exists in the fertilizer_data columns
        if nutrient_name in fertilizer_data.columns:
            # Search the fertilizer data for products that can fulfill this deficiency
            fertilizer_rows = fertilizer_data[fertilizer_data[nutrient_name] > 0]
            fertilizer_rows = fertilizer_rows.sort_values(by=nutrient_name, ascending=False)
            
            # Add the top 2 fertilizers to the recommended list
            for index, row in fertilizer_rows.head(2).iterrows():
                recommended_fertilizers.append((nutrient_name.replace('fert_', ''), row['fert_name'], row[nutrient_name]))
        else:
            print(f"No fertilizers found for {nutrient_name.replace('fert_', '')}")
    
    return recommended_fertilizers

def select_top_fertilizers(recommended_fertilizers):
    fertilizers_by_nutrient = {}

    for fertilizer in recommended_fertilizers:
        nutrient = fertilizer[0]
        if nutrient not in fertilizers_by_nutrient:
            fertilizers_by_nutrient[nutrient] = []
        fertilizers_by_nutrient[nutrient].append(fertilizer)

    top_fertilizers = []
    for nutrient_fertilizers in fertilizers_by_nutrient.values():
        sorted_fertilizers = sorted(nutrient_fertilizers, key=lambda x: x[2], reverse=True)
        top_fertilizers.append(sorted_fertilizers[0])
        if len(top_fertilizers) >= 5:
            break

    return top_fertilizers

def main():
    crop_name = "chestnutmushrooms"
    crop_growth_stage = "flowering"
    water_nutrition = [22.285, 281.586, 4.888, 41.023, 16.563, 7.896, 3.100, 1.883, 0.311, 0.297, 0.299, 0.317, 0.309, 0.297, 0.298, 0.298, 0.311, 0.290, 0.542, 1.417, 480.318]
    
    nutrient_deficiencies = calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition)
    recommended_fertilizers = recommend_fertilizers(nutrient_deficiencies)
    
    top_fertilizers = select_top_fertilizers(recommended_fertilizers)
    
    print("Top 5 Fertilizers:")
    for fertilizer in top_fertilizers:
        print(f"{fertilizer[0]}: {fertilizer[1]} ({fertilizer[2]})")

if __name__ == "__main__":
    main()

Top 5 Fertilizers:
nitrogen: urea (812)
phosphorus: nickelphosphate (773)
potassium: sugarcanemolasses (797)
calcium: calciumhydroxide (544)
magnesium: magnesiumoxide (602)


In [53]:
import pandas as pd

# Load the crop data
crop_data = pd.read_csv('crop_data.csv')

# Load the fertilizer data
fertilizer_data = pd.read_csv('pro_fert_data.csv')

def get_nutrient_name(index):
    nutrient_names = ['crop_nitrogen', 'crop_phosphorus', 'crop_potassium', 'crop_calcium', 'crop_magnesium', 'crop_sulfur', 'crop_copper', 'crop_chlorine', 'crop_boron', 'crop_iron', 'crop_zinc', 'crop_manganese', 'crop_molybdenum', 'crop_nickel']
    return nutrient_names[index]

def calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition):
    # Get the recommended nutrition values for the crop
    crop_row = crop_data[(crop_data['crop_name'] == crop_name) & (crop_data['crop_growth_stage'] == crop_growth_stage)]
    
    if crop_row.empty:
        print(f"No data found for {crop_name} in {crop_growth_stage} stage")
        return []
    
    recommended_nutrition = crop_row.iloc[0, 5:].values  # Adjust if your dataset has different columns
    
    # Convert recommended_nutrition to a pandas Series and then ensure it is numeric
    recommended_nutrition = pd.Series(recommended_nutrition)
    recommended_nutrition = pd.to_numeric(recommended_nutrition, errors='coerce').fillna(0).tolist()
    
    # Calculate the difference between user-input water nutrition values and recommended values
    nutrient_deficiencies = []
    for i in range(14):
        if isinstance(recommended_nutrition[i], (int, float)) and isinstance(water_nutrition[i], (int, float)):
            deficiency = recommended_nutrition[i] - water_nutrition[i]
            nutrient_deficiencies.append((i, deficiency))
        else:
            nutrient_deficiencies.append((i, "Invalid input"))
    
    # Sort the deficiencies by absolute value in descending order
    numeric_deficiencies = [x for x in nutrient_deficiencies if isinstance(x[1], (int, float))]
    numeric_deficiencies.sort(key=lambda x: abs(x[1]), reverse=True)
    non_numeric_deficiencies = [x for x in nutrient_deficiencies if not isinstance(x[1], (int, float))]
    nutrient_deficiencies = numeric_deficiencies + non_numeric_deficiencies
    
    # Return the top 5 nutrient deficiencies
    return nutrient_deficiencies[:5]

def recommend_fertilizers(nutrient_deficiencies):
    recommended_fertilizers = []
    for deficiency in nutrient_deficiencies:
        nutrient_name = get_nutrient_name(deficiency[0]).replace('crop_', 'fert_')
        deficiency_value = deficiency[1]
        
        # Check if the nutrient_name exists in the fertilizer_data columns
        if nutrient_name in fertilizer_data.columns:
            # Search the fertilizer data for products that can fulfill this deficiency
            fertilizer_rows = fertilizer_data[fertilizer_data[nutrient_name] > 0]
            fertilizer_rows = fertilizer_rows.sort_values(by=nutrient_name, ascending=False)
            
            # Add the top 2 fertilizers to the recommended list
            for index, row in fertilizer_rows.head(2).iterrows():
                recommended_fertilizers.append((nutrient_name.replace('fert_', ''), row['fert_name'], row[nutrient_name]))
        else:
            print(f"No fertilizers found for {nutrient_name.replace('fert_', '')}")
    
    return recommended_fertilizers

def select_top_fertilizers(recommended_fertilizers):
    fertilizers_by_nutrient = {}

    for fertilizer in recommended_fertilizers:
        nutrient = fertilizer[0]
        if nutrient not in fertilizers_by_nutrient:
            fertilizers_by_nutrient[nutrient] = []
        fertilizers_by_nutrient[nutrient].append(fertilizer)

    top_fertilizers = []
    for nutrient_fertilizers in fertilizers_by_nutrient.values():
        sorted_fertilizers = sorted(nutrient_fertilizers, key=lambda x: x[2], reverse=True)
        top_fertilizers.append(sorted_fertilizers[0])
        if len(top_fertilizers) >= 5:
            break

    return top_fertilizers

def main():
    crop_name = "armeniancucumbers"
    crop_growth_stage = "vegetative"
    water_nutrition = [22.285, 281.586, 4.888, 41.023, 16.563, 7.896, 3.100, 1.883, 0.311, 0.297, 0.299, 0.317, 0.309, 0.297]

    nutrient_deficiencies = calculate_nutrient_deficiencies(crop_name, crop_growth_stage, water_nutrition)
    recommended_fertilizers = recommend_fertilizers(nutrient_deficiencies)
    
    top_fertilizers = select_top_fertilizers(recommended_fertilizers)
    
    print("Top 5 Fertilizers:")
    for fertilizer in top_fertilizers:
        print(f"{fertilizer[0]}: {fertilizer[1]} ({fertilizer[2]})")

if __name__ == "__main__":
    main()


Top 5 Fertilizers:
phosphorus: nickelphosphate (773)
nitrogen: urea (812)
calcium: calciumhydroxide (544)
potassium: sugarcanemolasses (797)
magnesium: magnesiumoxide (602)


In [62]:
import pandas as pd
import random

# Load crop data and fertilizer data
crop_data = pd.read_csv('./crop_data.csv')
fertilizer_data = pd.read_csv('./pro_fert_data.csv')

def get_crop_requirements(crop_name, crop_growth_stage):
    crop = crop_data[(crop_data['crop_name'] == crop_name) & (crop_data['crop_growth_stage'] == crop_growth_stage)]
    if crop.empty:
        raise ValueError("Crop name or growth stage not found in the dataset.")
    return crop.iloc[0]

def get_deficiency_fertilizers(crop_requirements, current_water_content):
    deficiencies = {}
    for nutrient in ['nitrogen', 'phosphorus', 'potassium', 'calcium', 'magnesium', 'sulfur', 'copper', 'chlorine', 'boron', 'iron', 'zinc', 'manganese', 'molybdenum', 'nickel', 'cobalt', 'sodium']:
        crop_nutrient = crop_requirements[f'crop_{nutrient}']
        water_nutrient = current_water_content[nutrient]
        if water_nutrient < crop_nutrient:
            deficiencies[nutrient] = crop_nutrient - water_nutrient
    
    recommendations = []
    for nutrient, deficiency in deficiencies.items():
        fert = fertilizer_data[(fertilizer_data[f'fert_{nutrient}'] > 0) & (fertilizer_data['fert_type'] != 'phandecstabilizer') & (fertilizer_data['fert_type'] != 'growthregulator')]
        if not fert.empty:
            recommendations.append(fert.iloc[0])
    
    return recommendations

def get_random_ph_ec_stabilizer():
    stabilizers = fertilizer_data[fertilizer_data['fert_type'] == 'phandecstabilizer']
    if stabilizers.empty:
        return None
    return stabilizers.sample(n=1).iloc[0]

def get_random_growth_regulator(crop_growth_stage):
    # Only recommend growth regulator if crop is in seeding stage
    if crop_growth_stage.lower() != 'seeding':
        return None
    
    regulators = fertilizer_data[fertilizer_data['fert_type'] == 'growthregulator']
    if regulators.empty:
        return None
    return regulators.sample(n=1).iloc[0]

def recommend_fertilizers(crop_name, crop_growth_stage, current_water_content):
    crop_requirements = get_crop_requirements(crop_name, crop_growth_stage)
    deficiency_fertilizers = get_deficiency_fertilizers(crop_requirements, current_water_content)
    
    # Get random pH and EC stabilizers
    ph_ec_stabilizer = get_random_ph_ec_stabilizer()
    ec_balancer = get_random_ph_ec_stabilizer()  # You can modify this to get a different random stabilizer
    
    # Get random growth regulator (only if crop is in seeding stage)
    growth_regulator = get_random_growth_regulator(crop_growth_stage)
    
    return {
        'deficiency_fertilizers': deficiency_fertilizers,
        'ph_ec_stabilizer': ph_ec_stabilizer,
        'ec_balancer': ec_balancer,
        'growth_regulator': growth_regulator
    }

# Example usage
crop_name = 'frenchsorrel'
crop_growth_stage = 'seeding'  # Example: 'seeding', 'vegetative', 'flowering', etc.
current_water_content = {
    'ph': 6.5,
    'ec': 1.2,
    'nitrogen': 10,
    'phosphorus': 5,
    'potassium': 10,
    'calcium': 5,
    'magnesium': 5,
    'sulfur': 5,
    'copper': 0.5,
    'chlorine': 0.5,
    'boron': 0.5,
    'iron': 0.5,
    'zinc': 0.5,
    'manganese': 0.5,
    'molybdenum': 0.05,
    'nickel': 0.05,
    'cobalt': 0.05,
    'sodium': 5
}

recommendations = recommend_fertilizers(crop_name, crop_growth_stage, current_water_content)

print("Fertilizers for nutrient deficiencies:")
for fert in recommendations['deficiency_fertilizers']:
    print(fert['fert_name'])

print("\nFertilizer for pH stabilization:")
if recommendations['ph_ec_stabilizer'] is not None:
    print(recommendations['ph_ec_stabilizer']['fert_name'])
else:
    print("No pH stabilizer found.")

print("\nFertilizer for EC balancing:")
if recommendations['ec_balancer'] is not None:
    print(recommendations['ec_balancer']['fert_name'])
else:
    print("No EC balancer found.")

print("\nFertilizer for growth regulation:")
if recommendations['growth_regulator'] is not None:
    print(recommendations['growth_regulator']['fert_name'])
else:
    print("No growth regulator found.")


Fertilizers for nutrient deficiencies:
magnesiumnitrate
ammoniumpolyphosphate
potassiumthiosulfate
calciummolybdate
nickelcarbonate
cobaltgluconate

Fertilizer for pH stabilization:
potassiumcarbonate

Fertilizer for EC balancing:
silicicacid

Fertilizer for growth regulation:
cytokinins


In [ ]:
project-folder/
│
├── app.py
│
├── Data/
│   ├── crop_data.
│   └── fertilizer.csv
├── models/
│   ├── trained_model.pkl
│   └── plant_disease_model.pth
├── static/
│   ├── css/
│   │    ├── bootstrap.css
│   │    ├── font-awesome.min.css
│   │    └── style.css
│   ├── images/
│   └── scripts/
│        └── cities.js
├── templates/
│   ├── about.html
│   ├── contact.html
│   ├── crop-result.html
│   ├── crop.html
│   ├── disease-result.html
│   ├── disease.html
│   ├── fertilizer-result.html
│   ├── fertilizer.html
│   ├── index.html
│   ├── layout.html
│   ├── test_components.html
│   └── try_again.html
└── utils/
    ├── disease.py
    ├── fertilizer.py
    ├── model.py # file for disease prediction
    └── predict.py # file for crop prediction